In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 54.7 MB/s 
     |████████████████████████████████| 6.8 MB 30.0 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
from random import randrange

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',return_dict = True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sample_file = open("/content/drive/MyDrive/reading.txt", "r") 
main_text = sample_file.readlines()
sample_file.close()

In [ ]:
main_text

['Bovid\n',
 'A bovid is any member of almost 140 species of ungulates belonging to the family Bovidae. The bovids are the largest family of hoofed mammals and are native to Africa, Europe, Asia, and North America. Members include antelope, bison, buffalo, cattle, sheep and goats. Bovids have mutually beneficial symbiotic relationships with bacteria and other microorganisms that allow the digestion of cellulose, the most abundant form of living terrestrial biomass, but one that is indigestible for many animals, including humans.\n',
 '\n',
 'Bovids are not so common in endemic insular faunas and are mainly recorded in Southeast Asia, Japan and some Mediterranean islands. Ely the late Miocene, the bovids rapidly diversified, leading to the creation of 70 new genera. This late Miocene radiation was partly because most bovids became adapted to more open, grassland habitats. Some species of bovid are solitary, but others live in large groups with complex social structures.\n',
 '\n',
 'All

In [ ]:
for m in main_text:
  if m == '\n':
    main_text.remove(m)

main_text

['Bovid\n',
 'A bovid is any member of almost 140 species of ungulates belonging to the family Bovidae. The bovids are the largest family of hoofed mammals and are native to Africa, Europe, Asia, and North America. Members include antelope, bison, buffalo, cattle, sheep and goats. Bovids have mutually beneficial symbiotic relationships with bacteria and other microorganisms that allow the digestion of cellulose, the most abundant form of living terrestrial biomass, but one that is indigestible for many animals, including humans.\n',
 'Bovids are not so common in endemic insular faunas and are mainly recorded in Southeast Asia, Japan and some Mediterranean islands. Ely the late Miocene, the bovids rapidly diversified, leading to the creation of 70 new genera. This late Miocene radiation was partly because most bovids became adapted to more open, grassland habitats. Some species of bovid are solitary, but others live in large groups with complex social structures.\n',
 'All bovids have t

In [ ]:
length = len(main_text)
length

10

In [ ]:
def replace(list_name, index1, index2):
  temp = list_name[index1]
  list_name[index1] = list_name[index2]
  list_name[index2] = temp

In [ ]:
verbs = [[] for i in range(length)]

for i in range(length):
  text = nltk.word_tokenize(main_text[i]);
  pos_tagged = nltk.pos_tag(text)
  for ele in pos_tagged:
    if(ele[1].startswith('VB') and len(ele[0])>3):
      verbs[i].append(ele[0])

In [ ]:
verbs

[[],
 ['belonging', 'include', 'have', 'allow', 'living', 'including'],
 ['recorded', 'diversified', 'leading', 'became', 'live'],
 ['have', 'show', 'weigh', 'stands'],
 ['united',
  'Being',
  'composed',
  'undigested',
  'regurgitated',
  'chewed',
  'bear',
  'reduced',
  'have',
  'called',
  'provides',
  'grip',
  'have',
  'walk',
  'touch',
  'horns',
  'measure',
  'have',
  'horns'],
 ['consisting', 'suggests', 'pronghorn', 'endangered', 'been', 'extinct'],
 ['defined',
  'used',
  'describe',
  'fall',
  'have',
  'includes',
  'contains',
  'found',
  'woodland',
  'have',
  'adapted',
  'living',
  'antelope',
  'live',
  'splayed',
  'enable',
  'walk'],
 ['consists',
  'referred',
  'reach',
  'became',
  'bighorn',
  'have',
  'been',
  'found',
  'survive'],
 ['belonging', 'brown', 'eating', 'consume', 'manage', 'capture'],
 ['composed',
  'flattened',
  'grows',
  'forming',
  'branched',
  'possessing',
  'hence',
  'being',
  'built',
  'running',
  'have',
  'cush

In [ ]:
new_verbs = [[] for i in range(length)]

for i in range(length):
  new_verbs[i] = list(dict.fromkeys(verbs[i]))

new_verbs

[[],
 ['belonging', 'include', 'have', 'allow', 'living', 'including'],
 ['recorded', 'diversified', 'leading', 'became', 'live'],
 ['have', 'show', 'weigh', 'stands'],
 ['united',
  'Being',
  'composed',
  'undigested',
  'regurgitated',
  'chewed',
  'bear',
  'reduced',
  'have',
  'called',
  'provides',
  'grip',
  'walk',
  'touch',
  'horns',
  'measure'],
 ['consisting', 'suggests', 'pronghorn', 'endangered', 'been', 'extinct'],
 ['defined',
  'used',
  'describe',
  'fall',
  'have',
  'includes',
  'contains',
  'found',
  'woodland',
  'adapted',
  'living',
  'antelope',
  'live',
  'splayed',
  'enable',
  'walk'],
 ['consists',
  'referred',
  'reach',
  'became',
  'bighorn',
  'have',
  'been',
  'found',
  'survive'],
 ['belonging', 'brown', 'eating', 'consume', 'manage', 'capture'],
 ['composed',
  'flattened',
  'grows',
  'forming',
  'branched',
  'possessing',
  'hence',
  'being',
  'built',
  'running',
  'have',
  'cushioned',
  'pointed',
  'help',
  'absorb'

In [ ]:
answers1 = [[] for i in range(length)]
labels1 = [[] for i in range(length)]
answers2 = [[] for i in range(length)]
labels2 = [[] for i in range(length)]
answers3 = [[] for i in range(length)]
labels3 = [[] for i in range(length)]

for i in range(length):

  for verb in verbs[i]:
      
    labels_one_sentence1 = []
    labels_one_sentence2 = []
    labels_one_sentence3 = []

    split_text = main_text[i].split(verb, 1)
    text1 = split_text[0] + tokenizer.mask_token + split_text[1]
      
    input = tokenizer.encode_plus(text1, return_tensors = "pt")
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)

    output = model(**input)
    logits = output.logits

    softmax = F.softmax(logits, dim = -1)
    mask_word = softmax[0, mask_index, :]

    top_10 = torch.topk(mask_word, 30522, dim = 1)[1][0]
    level1 = top_10[0:(4**2)-1]
    level2 = top_10[4**2:(4**4)-1]
    level3 = top_10[(4**4):(4**6)-1]

    #level_1
    if tokenizer.encode_plus(verb).input_ids[1] in level1:
      answers1[i].append(verb)
      labels_one_sentence1.append(verb)
      options = torch.topk(mask_word, 4, dim = 1)[1][0]
      for token in options:
        word = tokenizer.decode([token])
        if (word!=verb):
          labels_one_sentence1.append(word)
      index = labels_one_sentence1.index(verb)
      correct_index = randrange(4)
      replace(labels_one_sentence1, correct_index, index)
      labels1[i].append(labels_one_sentence1)
    #level_2
    elif tokenizer.encode_plus(verb).input_ids[1] in level2:
      answers2[i].append(verb)
      labels_one_sentence2.append(verb)
      options = torch.topk(mask_word, 3, dim = 1)[1][0]
      for token in options:
        word = tokenizer.decode([token])
        labels_one_sentence2.append(word)
      correct_index = randrange(4)
      replace(labels_one_sentence2, correct_index, 0)
      labels2[i].append(labels_one_sentence2)
    #level_3
    elif tokenizer.encode_plus(verb).input_ids[1] in level3:
      answers3[i].append(verb)
      labels_one_sentence3.append(verb)
      options = torch.topk(mask_word, 4, dim = 1)[1][0]
      for token in options:
        word = tokenizer.decode([token])
        labels_one_sentence3.append(word)
      correct_index = randrange(4)
      replace(labels_one_sentence3, correct_index, 0)
      labels3[i].append(labels_one_sentence3)

In [ ]:
dictionary = {"sentences": [{} for i in range(length*3)]}

for i in range(length):
  #level_1
  if(len(answers1[i]) != 0):
    dictionary["sentences"][i].update({"text": main_text[i]})
    dictionary["sentences"][i].update({"level": "Basic"})
    dictionary["sentences"][i].update({"blank": [{} for j in range(len(answers1[i]))]})
    for j in range(len(answers1[i])):
      dictionary["sentences"][i]["blank"][j].update({"correct": answers1[i][j]})
      pos = dictionary["sentences"][i]["text"].index(answers1[i][j])
      dictionary["sentences"][i]["blank"][j].update({"pos": pos})
      dictionary["sentences"][i]["blank"][j].update({"words": labels1[i][j]})
      dictionary["sentences"][i].update({"text": dictionary["sentences"][i]["text"].replace(answers1[i][j], " ", 1)})
  #level_2
  if(len(answers2[i]) != 0):
    k = i+length
    dictionary["sentences"][k].update({"text": main_text[i]})
    dictionary["sentences"][k].update({"level": "Intermediate"})
    dictionary["sentences"][k].update({"blank": [{} for j in range(len(answers2[i]))]})
    for j in range(len(answers2[i])):
      dictionary["sentences"][k]["blank"][j].update({"correct": answers2[i][j]})
      pos = dictionary["sentences"][k]["text"].index(answers2[i][j])
      dictionary["sentences"][k]["blank"][j].update({"pos": pos})
      dictionary["sentences"][k]["blank"][j].update({"words": labels2[i][j]})
      dictionary["sentences"][k].update({"text": dictionary["sentences"][k]["text"].replace(answers2[i][j], " ", 1)})
  #level_3
  if(len(answers3[i]) != 0):
    k = i + length*2
    dictionary["sentences"][k].update({"text": main_text[i]})
    dictionary["sentences"][k].update({"level": "Advanced"})
    dictionary["sentences"][k].update({"blank": [{} for j in range(len(answers3[i]))]})
    for j in range(len(answers3[i])):
      dictionary["sentences"][k]["blank"][j].update({"correct": answers3[i][j]})
      pos = dictionary["sentences"][k]["text"].index(answers3[i][j])
      dictionary["sentences"][k]["blank"][j].update({"pos": pos})
      dictionary["sentences"][k]["blank"][j].update({"words": labels3[i][j]})
      dictionary["sentences"][k].update({"text": dictionary["sentences"][k]["text"].replace(answers3[i][j], " ", 1)})

In [ ]:
dictionary

{'sentences': [{},
  {'blank': [{'correct': 'belonging',
     'pos': 57,
     'words': ['belonging', 'belong', 'assigned', 'belongs']},
    {'correct': 'include',
     'pos': 201,
     'words': ['include', 'are', 'includes', 'included']},
    {'correct': 'have',
     'pos': 261,
     'words': ['form', 'have', 'maintain', 'develop']},
    {'correct': 'allow',
     'pos': 351,
     'words': ['allow', 'facilitate', 'control', 'promote', 'require']},
    {'correct': 'living',
     'pos': 407,
     'words': ['living', 'the', 'all', 'their', 'any']},
    {'correct': 'including',
     'pos': 477,
     'words': ['especially', 'including', 'particularly', 'notably']}],
   'level': 'Basic',
   'text': 'A bovid is any member of almost 140 species of ungulates   to the family Bovidae. The bovids are the largest family of hoofed mammals and are native to Africa, Europe, Asia, and North America. Members   antelope, bison, buffalo, cattle, sheep and goats. Bovids   mutually beneficial symbiotic relat

In [ ]:
for d in dictionary["sentences"]:
  if len(d)==0:
    dictionary["sentences"].remove(d)

In [ ]:
for d in dictionary["sentences"]:
  print(d)

{'text': 'A bovid is any member of almost 140 species of ungulates   to the family Bovidae. The bovids are the largest family of hoofed mammals and are native to Africa, Europe, Asia, and North America. Members   antelope, bison, buffalo, cattle, sheep and goats. Bovids   mutually beneficial symbiotic relationships with bacteria and other microorganisms that   the digestion of cellulose, the most abundant form of   terrestrial biomass, but one that is indigestible for many animals,   humans.\n', 'level': 'Basic', 'blank': [{'correct': 'belonging', 'pos': 57, 'words': ['belonging', 'belong', 'assigned', 'belongs']}, {'correct': 'include', 'pos': 201, 'words': ['include', 'are', 'includes', 'included']}, {'correct': 'have', 'pos': 261, 'words': ['form', 'have', 'maintain', 'develop']}, {'correct': 'allow', 'pos': 351, 'words': ['allow', 'facilitate', 'control', 'promote', 'require']}, {'correct': 'living', 'pos': 407, 'words': ['living', 'the', 'all', 'their', 'any']}, {'correct': 'inclu

In [ ]:
import json
json_object = json.dumps(dictionary, indent=len(dictionary))
  
# Writing to input.json
with open("input.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
json_file = open("input.json", "r")
print(json_file.read())

{
 "sentences": [
  {
   "text": "A bovid is any member of almost 140 species of ungulates   to the family Bovidae. The bovids are the largest family of hoofed mammals and are native to Africa, Europe, Asia, and North America. Members   antelope, bison, buffalo, cattle, sheep and goats. Bovids   mutually beneficial symbiotic relationships with bacteria and other microorganisms that   the digestion of cellulose, the most abundant form of   terrestrial biomass, but one that is indigestible for many animals,   humans.\n",
   "level": "Basic",
   "blank": [
    {
     "correct": "belonging",
     "pos": 57,
     "words": [
      "belonging",
      "belong",
      "assigned",
      "belongs"
     ]
    },
    {
     "correct": "include",
     "pos": 201,
     "words": [
      "include",
      "are",
      "includes",
      "included"
     ]
    },
    {
     "correct": "have",
     "pos": 261,
     "words": [
      "form",
      "have",
      "maintain",
      "develop"
     ]
    },
    {


In [ ]:
json_file = open("/content/drive/MyDrive/input.json", "r+") 
print(json_file.read())
json_file.close()

{
    "sentences": [
        {
            "text": "National Australia Bank's earnings prospects will rely on an ongoing strength of business and corporate banking, which jointly contributed 65% of its 1H pretax profit. These units showed stable net interest margins, offsetting a weakness in consumer banking. Market income also supports its revenue, as market volatility fuels demand for risk management and treasury business. Reduced credit costs bode well for its earnings too. Separately, NAB and peers' profit will suffer from the proposed 6-bp levy on liabilities.",
            "level": "Intermediate",
            "blank": [
                {
                "correct": "rely",
                "pos": 50,
                "words": [
                "rely",
                "depend",
                "rest",
                "improve"
                ]
                },
                {
                "correct": "showed",
                "pos": 180,
                "words": [
            